<h1 style='background:#F9EEEC; border:0; color: #EA7B67'><center>Pseudo Labeing</center></h1>

**This Kernel translated into Korean. If you not Korean, Check : <a href="https://www.kaggle.com/code/cdeotte/pseudo-labeling-qda-0-969#Step-3-&-4---Add-pseudo-label-data-and-build-second-model">LINK</a>**

<h1 style='background:#F9EEEC; border:0; color: #EA7B67'><center>TABLE OF CONTENTS</center></h1>

[1. Pseudo Labeling with QDA scores LB](#1)
    
[2. What is Pseudo Labeling?](#2)    

[3. Why does Pseudo Labeling work?](#3)        

[4. Step 1 and 2 - Build first QDA model and predict test](#4)  

[5. Step 3 & 4 - Add pseudo label data and build second model](#5)    

[6. Submit Predictions](#6)  

[7. Conclusion](#7)

<h1 style='background:#F9EEEC; border:0; color: #EA7B67'><center>START</center></h1>

<a id="1"></a>
# **<span style="color:#4686C8;">Pseudo Labeling with QDA scores LB</span>**

**"Instant Gratification" Competition에서는, 512개의 행과 40개의 열로 이루어진 512개의 모델이 있습니다. 40개의 변수가 있는 quadratic model은 더 많은 행을 원합니다.**

**Pseudo Labeling을 사용한다면, 400개의 행을 얻을 수 있으며, LB 와 PB의 성능을 향상시킬 수 있습니다.**


<a id="2"></a>
# **<span style="color:#4686C8;">What is Pseudo Labeling?</span>**

**Pseudo Labeling은 test data에서 예측한 것들을 trianing data에 추가하는 과정입니다.**

**Pseudo Labeling은 5가지의 절차에 따라 수행됩니다.**

**- 1. Training Data를 가지고, 모델을 생성합니다.**

**- 2. test data에 대해 예측합니다.**

**- 3. Training Data에 확신을 가진 test data를 추가합니다.**

**- 4. 결합된 데이터를 가지고, 새로운 모델을 생성합니다.**

**- 5. 새로운 모델로 test data를 예측한 후, Kaggle에 제출합니다.**

**Image로 과정을 확인하고 싶다면, <a href="https://www.kaggle.com/code/cdeotte/pseudo-labeling-qda-0-969#Why-does-Pseudo-Labeling-work?">LINK</a>를 참고하시기 바랍니다.**

<a id="3"></a>
# **<span style="color:#4686C8;">Why does Pseudo Labeling work?</span>**

**<a href="https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/89003">LINK</a>에서 Pseudo Labeling을 활용하여, 1등이 되었을 때, 매우 감격스러웠다.**

**이전에 학습된 모델에 의해 레이블이 지정된 알 수 없는 데이터로 훈련하면 동일한 모델이 어떻게 향상됩니까?**

**모델이 이미 정보를 알고 있기에 가능한 것이 아니라, 모델이 예측하였기 때문에 가능한 것입니다.**

## Load Data

In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score

train = pd.read_csv('../input/instant-gratification/train.csv')
test = pd.read_csv('../input/instant-gratification/test.csv')

train.head()

<a id="4"></a>
# **<span style="color:#4686C8;">Step 1 and 2 - Build first QDA model and predict test</span>**

In [22]:
cols = [c for c in train.columns if c not in ['id', 'target']]
cols.remove('wheezy-copper-turtle-magic')

oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in range(512):
    train2 = train[train['wheezy-copper-turtle-magic'] == i]
    test2 = test[test['wheezy-copper-turtle-magic'] == i]
    
    # 먼저 같은 데이터만을 가져옵니다.
    if (len(train2) == 0) & (len(test2) == 0):
        continue
    
    # Train, Test 2가지로 분류해줍니다.
    idx1 = train2.index
    idx2 = test2.index
    
    # index 초기화
    train2.reset_index(drop = True, inplace = True)
    
    # Feature Selection
    sel = VarianceThreshold(threshold = 1.5).fit(train2[cols])
    train3 = sel.transform(train2[cols])
    
    if len(test2) > 0:
        test3 = sel.transform(test2[cols])
    
    skf = StratifiedKFold(n_splits = 11, random_state = 42, shuffle = True)
    
    # wheezy-copper-turtle-magic와 값이 같은 것을 고르면 됩니다.
    
    for train_index, test_index in skf.split(train3, train2['target']):
        clf = QuadraticDiscriminantAnalysis(reg_param=0.5)
        clf.fit(train3[train_index, :], train2.loc[train_index]['target'])
        oof[idx1[test_index]] = clf.predict_proba(train3[test_index, :])[:, 1]
        if len(test2) > 0:
            preds[idx2] += clf.predict_proba(test3)[:, 1] / skf.n_splits
        
auc = roc_auc_score(train['target'], oof)
print('QDA scores CV = ', round(auc, 5))

<a id="5"></a>
# **<span style="color:#4686C8;">Step 3 & 4 - Add pseudo label data and build second model</span>**

In [34]:
test['target'] = preds
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for k in range(512):
    train2 = train[train['wheezy-copper-turtle-magic'] == k]
    train2p = train2.copy()
    idx1 = train2.index

    test2 = test[test['wheezy-copper-turtle-magic'] == k]
    test2p = test2[(test2['target'] <= 0.01) | (test2['target'] >= 0.99)].copy()
    
    test2p.loc[test2p['target'] >= 0.5, 'target'] = 1
    test2p.loc[test2p['target'] < 0.5, 'target'] = 0
    
    train2p = pd.concat([train2p, test2p], axis = 0)
    train2p.reset_index(drop = True, inplace = True)
    
    sel = VarianceThreshold(threshold = 1.5).fit(train2p[cols])
    train3p = sel.transform(train2p[cols])
    train3 = sel.transform(train2[cols])
    
    if len(test2) > 0:
        test3 = sel.transform(test2[cols])
    
    skf = StratifiedKFold(n_splits = 11, random_state = 42, shuffle = True)
    for train_index, test_index in skf.split(train3p, train2p['target']):
        test_index3 = test_index[test_index<len(train3)]
        
        clf = QuadraticDiscriminantAnalysis(reg_param = 0.5)
        clf.fit(train3p[train_index, :], train2p.loc[train_index]['target'])
        oof[idx1[test_index3]] = clf.predict_proba(train3[test_index3, :])[:, 1]
        
        if len(test2) > 0:
            preds[test2.index] += clf.predict_proba(test3)[:, 1] / skf.n_splits

auc = roc_auc_score(train['target'],oof)
print('Pseudo Labeled QDA scores CV =',round(auc,5))

<a id="6"></a>
# **<span style="color:#4686C8;">Submit Predictions</span>**

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission.csv',index=False)

import matplotlib.pyplot as plt
plt.hist(preds,bins=100)
plt.title('Final Test.csv predictions')
plt.show()

<a id="7"></a>
# **<span style="color:#4686C8;">Conclusion</span>**

**요번 시간에는 Pseudo Labeling이 무엇인지, 어떻게 작동하는지 배웠습니다. 해당 대회를 통하여, Pseudo Labeling을 사용하는 경우 0.005가 개선되는 것을 알 수 있었습니다.**

**해당 대회에 Solution을 제출하는 경우, 공개 데이터인 test와 비공개 데이터를 모두 합쳐 작동하기 때문에, 데이터 양이 2배 더 많아집니다.**